In [836]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil

import numpy as np

In [837]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [838]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def cost(state):
    return len(state.taken)

def distance(state, action):

    taken_sets=list(state.taken)
    print("taken sets", taken_sets)
    #print("action",action)
    currently_covered=reduce(
        np.logical_or,
        [SETS[i] for i in taken_sets],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )
    #print("currently_covered",currently_covered)
 
    if action:
        newly_covered=SETS[action]
        #print("newly_covered", newly_covered)
        difference=0
        for i in range(len(newly_covered)):
            if (newly_covered[i] == currently_covered[i] ):
                difference+=1
        #print("difference",difference)
        overlap=difference
        taken_sets.append(action)
        

    else:
        overlap=0
        
    distance=PROBLEM_SIZE - sum(
            reduce(
                np.logical_or,
                [SETS[i] for i in taken_sets],
                np.array([False for _ in range(PROBLEM_SIZE)]),
            ))
    #print("overlap",overlap)
    print("distance",distance)
    return overlap*distance

def h(state):
    
    already_covered = reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )
   
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)

    new_metric=max(sum(np.logical_not(np.logical_xor(SETS[s], already_covered))) for s in state.not_taken)

    return optimistic_estimate*0.9+0.1*new_metric
    #return optimistic_estimate
def f(state):
    return h(state)+cost(state)


In [839]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

In [840]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
#print("initial current state", current_state)
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        #print("new state", new_state)
        frontier.put((f(new_state), new_state))
   
    _, current_state = frontier.get()
    #print("current state",current_state)

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 2 steps (2 tiles)


In [841]:
current_state

State(taken={3, 6}, not_taken={0, 1, 2, 4, 5, 7, 8, 9})

In [842]:
for i in range(len(SETS)):
    print(i)
    print(SETS[i])

0
[False False False False  True]
1
[False False False False False]
2
[False False False False  True]
3
[False  True  True False False]
4
[ True False False False False]
5
[False False False False False]
6
[ True False False  True  True]
7
[False False False  True  True]
8
[False False  True  True False]
9
[ True  True False False False]
